In [16]:
!pip install kaggle

In [17]:
!mkdir -p ~/.kaggle/

In [18]:
!cp /content/kaggle.json ~/.kaggle/

In [19]:
!chmod 600 ~/.kaggle/kaggle.json

In [20]:
!kaggle competitions download -c dogs-vs-cats

100% 810M/812M [00:06<00:00, 218MB/s]
100% 812M/812M [00:06<00:00, 137MB/s]


In [21]:
!unzip -q dogs-vs-cats.zip -d /content/
!unzip -q /content/train.zip -d /content/train_data/

In [22]:
!rm /content/dogs-vs-cats.zip
!rm /content/train.zip
!rm /content/test1.zip

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16 # Import the VGG16 model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [24]:
import os
import shutil

In [25]:
source_dir = '/content/train_data/train/'

In [26]:

cats_dir = os.path.join(source_dir, 'cats')
dogs_dir = os.path.join(source_dir, 'dogs')

os.makedirs(cats_dir, exist_ok=True)
os.makedirs(dogs_dir, exist_ok=True)

In [27]:
for filename in os.listdir(source_dir):
    if filename.startswith('cat.'):
        shutil.move(os.path.join(source_dir, filename), os.path.join(cats_dir, filename))
    elif filename.startswith('dog.'):
        shutil.move(os.path.join(source_dir, filename), os.path.join(dogs_dir, filename))

In [28]:
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values from 0-255 to 0-1
    rotation_range=20,       # Random rotations (degrees)
    width_shift_range=0.2,   # Random horizontal shifts (fraction of total width)
    height_shift_range=0.2,  # Random vertical shifts (fraction of total height)
    shear_range=0.2,         # Shear transformations
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Random horizontal flips
    fill_mode='nearest',     # Strategy for filling in new pixels created by transformations
    validation_split=0.2     # Allocate 20% of the data for validation
)

In [29]:
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [30]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

NameError: name 'data_dir' is not defined

In [31]:
data_dir = '/content/train_data/train'

In [32]:
image_size = (224, 224)
batch_size = 32

In [33]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [34]:
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

Found 20000 images belonging to 2 classes.


In [35]:
validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

Found 5000 images belonging to 2 classes.


In [36]:
print(f"Classes detected by generators: {train_generator.class_indices}")

Classes detected by generators: {'cats': 0, 'dogs': 1}


In [37]:
# Load VGG16 pre-trained on ImageNet, without its top (classification) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [38]:
for layer in base_model.layers:
    layer.trainable = False

In [39]:
x = base_model.output
x = Flatten()(x) # Flatten the output from the VGG16 convolutional base
x = Dense(256, activation='relu')(x) # A new dense layer
x = Dropout(0.5)(x) # Dropout for regularization
predictions = Dense(1, activation='sigmoid')(x) # Output layer for binary classification

In [40]:
model = Model(inputs=base_model.input, outputs=predictions)

In [41]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [42]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [44]:
epochs = 10 # Start with fewer epochs for transfer learning, as it converges faster

print(f"\nStarting VGG16 transfer learning model training for {epochs} epochs...")




Starting VGG16 transfer learning model training for 10 epochs...


In [45]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 295s 472ms/step - accuracy: 0.7826 - loss: 0.4480 - val_accuracy: 0.9184 - val_loss: 0.2026
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 291s 465ms/step - accuracy: 0.8745 - loss: 0.2823 - val_accuracy: 0.9262 - val_loss: 0.1779
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 293s 469ms/step - accuracy: 0.8850 - loss: 0.2645 - val_accuracy: 0.9288 - val_loss: 0.1762
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 305s 489ms/step - accuracy: 0.8965 - loss: 0.2442 - val_accuracy: 0.9308 - val_loss: 0.1640
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 292s 466ms/step - accuracy: 0.8983 - loss: 0.2383 - val_accuracy: 0.8974 - val_loss: 0.2300
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 290s 463ms/step - accuracy: 0.9020 - loss: 0.2329 - val_accuracy: 0.9326 - val_loss: 0.1691
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 292s 467ms/step - accuracy: 0.9065 - loss: 0.2216 - val_accuracy: 0.9310 - val_loss: 0.1678
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 295s 471ms/step - accuracy: 0.9046 -

In [46]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [47]:
checkpoint_filepath = 'best_vgg16_model.h5' # This will be saved in your Colab session's /content/ directory
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False, # Save the entire model
    monitor='val_accuracy',  # Monitor validation accuracy
    mode='max',              # Maximize validation accuracy
    save_best_only=True,     # Only save the best model
    verbose=1
)

In [48]:
early_stopping_callback = EarlyStopping(
    monitor='val_loss', # Monitor validation loss
    patience=5,         # Number of epochs with no improvement after which training will be stopped
    mode='min',         # Minimize validation loss
    verbose=1,
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity.
)

I left the code until here since my computer got very slowdown